In [1]:
import json

with open('data/myip.json') as f:
    myip_domains_dict = json.load(f)

In [2]:
domains = myip_domains_dict['list']
urls = ['http://'+domain for domain in domains]

In [3]:
from tld import get_tld
from urllib.parse import urlparse

In [333]:
url = urls[1]
url

'http://2ip.tools'

In [334]:
get_tld(url)

'tools'

In [336]:
import tldextract

def remove_tld(domain):
    ext = tldextract.extract(domain)
    if ext.subdomain:
        return '.'.join(ext[:2])
    return ext.domain

domains_no_tld = set(remove_tld(domain) for domain in domains)
# domains_no_tld

In [388]:
urls

['http://2ip.ru',
 'http://2ip.tools',
 'http://aboutmyip.com',
 'http://api.ipify.org',
 'http://api.myip.com',
 'http://api.wipmania.com',
 'http://bearsmyip.com',
 'http://bot.whatismyipaddress.com',
 'http://check-my-ip.net',
 'http://checkip-waw.dyndns.com',
 'http://checkip.amazonaws.com',
 'http://checkip.dns.he.net',
 'http://checkip.dyndns.com',
 'http://checkip.dyndns.es',
 'http://checkip.dyndns.org',
 'http://checkip.narak.com',
 'http://checkmyip.com',
 'http://cmyip.com',
 'http://cmyip.net',
 'http://crymyip.com',
 'http://curlmyip.com',
 'http://curlmyip.net',
 'http://dawhois.com',
 'http://db-ip.com',
 'http://dnswatch.info',
 'http://dpool.sina.com.cn',
 'http://e-localizaip.com',
 'http://extreme-ip-lookup.com',
 'http://findmyip.org',
 'http://findmyipaddress.com',
 'http://formyip.com',
 'http://freegeoip.app',
 'http://freegeoip.live',
 'http://geodatatool.com',
 'http://geoip.co.uk',
 'http://geoip.vmn.net',
 'http://geoiptool.com',
 'http://get-myip.com',
 'htt

In [337]:
from rds import redis_connect, redis_get

client = redis_connect()

In [338]:
client.dbsize()

11403506

In [365]:
def scan_keys(r, pattern):
    "Returns a list of all the keys matching a given pattern"

    cur, keys  = r.scan(cursor=0, match=pattern, count=2)
    
    return result

In [370]:
client.scan(cursor=4509696, count=2000)

(9398272,
 [b'sv01.bit.telesport.group',
  b'r6.sn-uxap5nvoxg5-ixhs.googlevideo.com',
  b'dhcp-128-107-163-56.cisco.com',
  b'dfc313bc093dfaff19866c301d216af9.safeframe.googlesyndication.com',
  b'dailycuteboys.tblog.com',
  b'pauls.renlife.ru',
  b'tmeic-ge.com',
  b'mail.dam-computers.com',
  b'ssl84483.cloudflare.com',
  b'bet-ecmo.fr',
  b'secure.cdn5.wdpromedia.com',
  b'volunteer.dsmviewer.ru',
  b'134.24.41.212.43tmjpd2ubw4hdy53anef6g4wy.zen.dq.spamhaus.net',
  b'cf7c82588febcbff44242e7b6e94a6fd.fp.measure.office.com',
  b'cronostv.site',
  b'moi-northeurope-hot.northeurope.cloudapp.azure.com',
  b'mail.tctr.ru',
  b'www.oficinafem.com',
  b'woolly.yookassa.ru',
  b'k185111.ppp.cpcpipe.ru',
  b'avast.jobs.cz',
  b'www.ibnuhasyim.com',
  b'1755476162.verify.yandex.ru',
  b'sezanm.cz',
  b'89985075.verify.yandex.ru',
  b'193-77.yookassa.ru',
  b'891.coll.ning.com',
  b'www.perf.yookassa.ru',
  b'yicding-lnx.cisco.com',
  b'world.bc.gapx.yahoodns.net',
  b's834b.silatv.ru',
  b'pan

In [339]:
import json

data_random = []
while len(data_random) < len(domains_no_tld) * 10:
    domain = client.randomkey().decode()
    data_random.append(remove_tld(domain))

In [340]:
X = data_random + list(domains_no_tld)
y = [0]*len(data_random) + [1]*len(domains_no_tld)

In [341]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [342]:
ngram_range = (1,5)
min_df = 0.05

tfidf = TfidfVectorizer(ngram_range=ngram_range, analyzer='char', min_df=min_df)
tfidf.fit(domains_no_tld)
print(len(tfidf.vocabulary_))
vocab = tfidf.vocabulary_

106


In [343]:
tfidf = TfidfVectorizer(ngram_range=ngram_range, analyzer='char', min_df=min_df, vocabulary=vocab)
rf = RandomForestClassifier(n_estimators=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test)

(1478, 370)

In [380]:
pipeline = Pipeline([
    ('tfidf', tfidf),
    ('classifier', rf)
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[334,   1],
       [ 11,  24]])

In [383]:
import joblib

joblib.dump(pipeline, '/Users/nikolayprudkovskiy/Projects/web_category_v2/models/myip_url.joblib')

['/Users/nikolayprudkovskiy/Projects/web_category_v2/models/myip_url.joblib']

In [351]:
# X_train

In [381]:
print(confusion_matrix(y_test, y_pred))

[[334   1]
 [ 11  24]]


In [359]:
data_random_test = []
while len(data_random_test) < len(domains_no_tld) * 100:
    domain = client.randomkey().decode()
    data_random_test.append((remove_tld(domain), domain))
len(data_random_test)

16800

In [360]:
# tfidf.vocabulary_

In [361]:
pipeline.predict([data for data, _ in data_random_test]).sum()

45

In [362]:
pipeline.predict(domains_no_tld).sum() / len(domains_no_tld)

0.9285714285714286

In [363]:
for d in domains_no_tld:
#     pred = pipeline.predict_proba([d])[:, 1]
    pred = pipeline.predict([d])
#     if pred <= 0.6:
    if pred == 0:
        print(d)

hostip
utrace
dnswatch
ifcfg
keliweb
ip.webmasterhome
showip
vinflag
icanhazip
ifconfig
ipv6-test
ip-info.ff.avast


In [364]:
for domain_no_tld, domain in data_random_test:
    pred = pipeline.predict_proba([domain_no_tld])[:, 1]
#     pred = pipeline.predict([domain_no_tld])
    if pred > 0.6:
        print(domain)

83o.no-ip.biz
kou92.no-ip.biz
chipportal.ru
clipbongda.info
easyiphoneunlocking.com
toip.nphr.ru
pixo00.no-ip.org
iputochkin.ru
ipaddress2web.telesport.group
7-zip.softonic.jp
4ip.info
snip.sb-mt.ru
sip.student.mcy-1.ru
tippnet.rs
mip-media.com
cn1833vip.awmdm.com
trolll.no-ip.org
goodwill-ip.ru
cheesethecakerecipes.com
iprojector.ru
roadgrip.co.uk
quick-ip-lookup.info
aitip.ru
jjl.one
sip.itstr.pro
